In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import random

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
X = np.reshape([random()*100 for i in range(400)],(100,4))

In [3]:
col_max = np.amax(X,axis=0)
col_min = np.amin(X,axis=0)
centroid = (col_max + col_min)/2

In [4]:
Z = (X-centroid)**2

In [5]:
alpha = (1.0 / 100) * np.ones(100)

In [6]:
ZT = Z.T
h = np.matmul(alpha,Z)